# Debug Credentials with Signature Provider

This notebook describes how to use a custom remote signing service for debug certificate generation

In [2]:
%run ../init_notebook.ipynb

import pprint
import os

pp = pprint.PrettyPrinter(indent=4)

plugins_dir = 'plugins/'

SASP_PLUGIN = os.path.join(plugins_dir, 'sasp.py')
WORKSPACE = "workspace/dat/" # change this to path to your workspace
DATA_DIR = "data_dat/" # change this to path to your workspace
VERBOSITY = "-v" # verbosity of commands, might be -v or -vv for debug or blank for no additional info

env: JUPYTER_SPSDK=1
Created `%!` as an alias for `%execute`.


### Signature Provider Plugin

First, we need to setup the Signature Provider plugin and start the custom HSM. In order to do that, open this [notebook](signature_provider.ipynb) and follow the instructions there. Once you are done, come back and continue here.

### Config File Setup

The nxpdebugmbox configuration file must be updated in order to integrate the custom Signature Provider.

The signature provider configuration must meet following rules:
  - Configuration key
    - key names `sign_provider` or `signProvider` are allowed

  - Configuration value
    - format `"type=<sp_type>;<key1>=<value1>;<key2>=<value2>;..."`
    - the `sp_type` has to match the sp_type class attribute defined in the custom signature provider(`plugins/sasp.py`)
    - the remaining key-value pairs are passed to the `__init__` method of the concrete Signature Provider
    - e.g.: `"type=file;file_path=private_key.pem"` will instantiate `spsdk.crypto.PlainFileSP(file_path='private_key.pem')`

New configuration field `rot_id` has been introduced:
    - due to the nature of creating Debug Credential file we need to know in advance which of the private keys will be used to perform the actual signing
    - `rot_id` is a 0-based index representing the private key that will be used with respect to `rot_meta`
    - e.g.: if we want to use a private key that corresponds to the public key `p1_cert0_2048.pub`, `rot_id` has to be set to `1`


In [3]:
import yaml
import os
import shutil
from spsdk.utils.misc import load_configuration
# choose family for the MCU
FAMILY = "rt5xx"
CONFIG_PATH = os.path.join(WORKSPACE, 'dck_rsa_2048.yml')

os.makedirs(WORKSPACE, exist_ok=True)
shutil.copyfile(os.path.join(DATA_DIR, 'dck_rsa_2048.yml'), CONFIG_PATH)

config = load_configuration(CONFIG_PATH)
del config['rotk']
config['sign_provider'] = "type=sasp;key_number=0"

with open(CONFIG_PATH, 'w') as file:
    yaml.dump(config, file, default_flow_style=False)

files_needed = ['p0_cert0_2048.pub', 'p1_cert0_2048.pub', 'dck.pub']
for file in files_needed:
    shutil.copyfile(os.path.join(DATA_DIR, file), os.path.join(WORKSPACE, file))

pp.pprint(f"All files are ready in folder '{WORKSPACE}'")


"All files are ready in folder 'workspace/dat/'"


### Execution

At this point, we have everything we need to run create debug certificate signed by remote HSM.

In [4]:
CONFIG_PATH = os.path.join(WORKSPACE, 'dck_rsa_2048.yml')
OUTPUT_PATH = os.path.join(WORKSPACE, 'dc_out.cert')
print(CONFIG_PATH)

%! nxpdebugmbox gendc --config $CONFIG_PATH --plugin $SASP_PLUGIN  $OUTPUT_PATH
 
# check if the signed image exists
assert os.path.exists(OUTPUT_PATH)

workspace/dat/dck_rsa_2048.yml


nxpdebugmbox gendc --config workspace/dat/dck_rsa_2048.yml --plugin plugins/sasp.py  workspace/dat/dc_out.cert 
RoT Key Hash: 5905022784a39901b0dc0860c9455cd1b83c5336a2e973825759961554664c89
Creating Debug credential file succeeded


### HSM teardown

Last step is to stop custom HSM. In order to do that, open again the [notebook](sahsm.ipynb) and stop the running jupyter notebook code cell.